In [1]:
import numpy as np
import os

layerList = ["가", "나", "다", "라", "마", "바", "아", "자", "차", "카", "타", "파", "하"]

all_x_array = []
all_y_array = []

for dir in layerList:
    fileList = os.listdir(f"../dataset_dir/image_npy(50x50)/image_npy/{dir}")

    for file in fileList:
        y_array = dict.fromkeys(layerList, 0) # 12000개의 str 값을 0으로 초기화하는 dict 생성

        if file[-8:-4] == "data":
            x_array = np.load(f"../dataset_dir/image_npy(50x50)/image_npy/{dir}/{file}")
            all_x_array.append(x_array.tolist())

            y_array[dir] = 1 # 사용자가 지정한 str 값을 1로 변경
            y_array = list(y_array.values())
            all_y_array.append(y_array)


In [2]:
x_array = np.array(all_x_array)
y_array = np.array(all_y_array)

In [3]:
print(x_array.shape)
print(y_array.shape)

(794, 2500)
(794, 13)


In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(
    x_array, y_array, test_size=0.2, shuffle=False, random_state=1004
)

In [5]:
X_train = X_train.astype("float32") / 255
Y_train = Y_train.astype("float32") / 255

X_test = X_test.astype("float32") / 255
Y_test = Y_test.astype("float32") / 255

In [6]:
print(X_train.shape)
print(X_test.shape)

print(Y_train.shape)
print(Y_test.shape)

(635, 2500)
(159, 2500)
(635, 13)
(159, 13)


In [7]:
u = np.unique(Y_train, axis=0)
len(u)

11

In [12]:
import tensorflow as tf
import keras

model = keras.Sequential([
    keras.Input(shape=(50, 50, 1)),
    keras.layers.Conv2D(32, (4, 4), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(13, activation="softmax"),
])

ValueError: Input 0 of layer "conv2d_4" is incompatible with the layer: expected min_ndim=4, found ndim=3. Full shape received: (None, 50, 50)

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 47, 47, 32)        544       
                                                                 
 max_pooling2d (MaxPooling2  (None, 23, 23, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 10, 10, 64)        0         
 g2D)                                                            
                                                                 
 flatten (Flatten)           (None, 6400)              0         
                                                                 
 dense (Dense)               (None, 128)               8

In [10]:
batch_size = 128
epochs = 15

model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.Adam(0.005), metrics=["accuracy"])

model.fit(X_train, Y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, Y_test))

Epoch 1/15


ValueError: in user code:

    File "/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py", line 1401, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py", line 1384, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py", line 1373, in run_step  **
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.11/dist-packages/keras/src/engine/training.py", line 1150, in train_step
        y_pred = self(x, training=True)
    File "/usr/local/lib/python3.11/dist-packages/keras/src/utils/traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "/usr/local/lib/python3.11/dist-packages/keras/src/engine/input_spec.py", line 298, in assert_input_compatibility
        raise ValueError(

    ValueError: In